In [1]:
import time
import requests
import json
from authlib.jose import JsonWebSignature
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os

In [2]:
# Configuracion para que pandas muestre todas las columnas
pd.set_option("display.max_columns", None)

# Configuración para mostrar números en notación normal
pd.set_option("display.float_format", "{:.0f}".format)

In [3]:
load_dotenv()  # Carga las variables del archivo .env

token_id = os.getenv("API_TOKEN_ID")
token_secret = os.getenv("API_TOKEN_SECRET")

In [4]:
# Base URL and endpoint
base_url = "https://api.absolute.com"
endpoint = "/v3/reporting/applications-advanced"
page_size = 500  # Tamaño de página
# Inicializar lista para almacenar todos los datos
all_data = []

# Paginación: token para la siguiente página
next_page = None

while True:
    # Preparar el query string
    query_string = f"pageSize={page_size}"
    if next_page:
        query_string += f"&nextPage={next_page}"

    # Crear la solicitud
    request = {
        "method": "GET",
        "contentType": "application/json",
        "uri": endpoint,
        "queryString": query_string,
        "payload": {}
    }

    # Crear el payload y firmarlo con JsonWebSignature
    request_payload_data = {
        "data": request["payload"]
    }
    headers = {
        "alg": "HS256",
        "kid": token_id,
        "method": request["method"],
        "content-type": request["contentType"],
        "uri": request["uri"],
        "query-string": request["queryString"],
        "issuedAt": round(time.time() * 1000)
    }

    jws = JsonWebSignature()
    signed = jws.serialize_compact(headers, json.dumps(request_payload_data), token_secret)

    # Enviar la solicitud
    request_url = f"{base_url}/jws/validate"
    response = requests.post(request_url, signed, {"content-type": "text/plain"})

    # Verificar la respuesta
    if response.status_code != 200:
        print(f"Error: {response.status_code}, {response.text}")
        break

    # Procesar los resultados
    result = response.json()
    all_data.extend(result["data"])

    # Obtener el token para la siguiente página
    next_page = result.get("metadata", {}).get("pagination", {}).get("nextPage")

    # Terminar si no hay más páginas
    if not next_page:
        break


In [5]:
# Transformar el json a una dataframe de pandas
df_applications = pd.DataFrame(all_data)

In [6]:
df_applications.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9781 entries, 0 to 9780
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   deviceUid               9781 non-null   object 
 1   accountUid              9781 non-null   object 
 2   appName                 9781 non-null   object 
 3   esn                     9781 non-null   object 
 4   deviceName              9781 non-null   object 
 5   deviceSerialNumber      9781 non-null   object 
 6   userName                9781 non-null   object 
 7   installPath             9781 non-null   object 
 8   installDateTimeUtc      9581 non-null   object 
 9   osName                  9781 non-null   object 
 10  major                   9738 non-null   float64
 11  minor                   9738 non-null   float64
 12  deviceAppId             9781 non-null   object 
 13  appId                   9781 non-null   object 
 14  appOriginalName         9781 non-null   

In [7]:
df_applications.head(3)

,deviceUid,accountUid,appName,esn,deviceName,deviceSerialNumber,userName,installPath,installDateTimeUtc,osName,major,minor,deviceAppId,appId,appOriginalName,appPublisher,appOriginalPublisher,appVersion,appOriginalVersion,firstDetectDateTimeUtc,lastScanDateTimeUtc
0,082bbc5e-506a-4688-82bc-e667aa17f3ac,fc39ea8e-61df-4330-aaca-a6ac74f0b025,TightVNC,6LJCEBX5M9AA4IEM0016,NB-CEM038,5CD42275ML,ARUNTNET\terrencio.cruz,c:\program files\tightvnc,2024-08-02T05:00:00Z,Microsoft Windows 11 Pro,2,8,40cfcbad-3d1a-35de-85c4-13a951d556a0,0005f8a9-26f2-3f3a-99b7-5ff6fd863459,TightVNC,GlavSoft,GlavSoft LLC.,2.8.81.0,2.8.81.0,2024-08-02T21:58:55.591Z,2024-12-20T15:31:15.735Z
1,109bdcbd-4147-4650-8753-a42ad2c6b691,fc39ea8e-61df-4330-aaca-a6ac74f0b025,TightVNC,6LJCEBX5M9AA4IEM0057,NB-CEM061,5CG4311L6S,ARUNTNET\toninho.serrano,c:\program files\tightvnc,2024-12-06T05:00:00Z,Microsoft Windows 11 Pro,2,8,5d45297c-9cb6-38e8-b773-95b49bc8cb2c,0005f8a9-26f2-3f3a-99b7-5ff6fd863459,TightVNC,GlavSoft,GlavSoft LLC.,2.8.81.0,2.8.81.0,2024-12-07T00:23:34.615Z,2024-12-20T20:07:20.935Z
2,16ce18eb-d73c-4829-9589-1807bf20bd60,fc39ea8e-61df-4330-aaca-a6ac74f0b025,TightVNC,6LJCEBX5M9AA4IEM0080,NB-CEM071,5CD43928QD,DESKTOP-TH3MTFL\user.tic,c:\program files\tightvnc,2024-12-20T05:00:00Z,Microsoft Windows 11 Pro,2,8,ea51acf6-4e61-37ef-a554-108e8529e083,0005f8a9-26f2-3f3a-99b7-5ff6fd863459,TightVNC,GlavSoft,GlavSoft LLC.,2.8.81.0,2.8.81.0,2024-12-20T17:48:19.289Z,2024-12-20T19:53:53.536Z


In [8]:
# Buscar filas duplicadas
df_applications.duplicated().sum()

0

In [9]:
#borrar columnas que no se van a necesitar
df_applications= df_applications.drop(['accountUid', 'esn','deviceName', 'userName', 'deviceAppId', 
                                       'appOriginalName', 'appOriginalPublisher', 'appOriginalVersion'] , axis=1) 

In [10]:
df_applications.head(3)

,deviceUid,appName,deviceSerialNumber,installPath,installDateTimeUtc,osName,major,minor,appId,appPublisher,appVersion,firstDetectDateTimeUtc,lastScanDateTimeUtc
0,082bbc5e-506a-4688-82bc-e667aa17f3ac,TightVNC,5CD42275ML,c:\program files\tightvnc,2024-08-02T05:00:00Z,Microsoft Windows 11 Pro,2,8,0005f8a9-26f2-3f3a-99b7-5ff6fd863459,GlavSoft,2.8.81.0,2024-08-02T21:58:55.591Z,2024-12-20T15:31:15.735Z
1,109bdcbd-4147-4650-8753-a42ad2c6b691,TightVNC,5CG4311L6S,c:\program files\tightvnc,2024-12-06T05:00:00Z,Microsoft Windows 11 Pro,2,8,0005f8a9-26f2-3f3a-99b7-5ff6fd863459,GlavSoft,2.8.81.0,2024-12-07T00:23:34.615Z,2024-12-20T20:07:20.935Z
2,16ce18eb-d73c-4829-9589-1807bf20bd60,TightVNC,5CD43928QD,c:\program files\tightvnc,2024-12-20T05:00:00Z,Microsoft Windows 11 Pro,2,8,0005f8a9-26f2-3f3a-99b7-5ff6fd863459,GlavSoft,2.8.81.0,2024-12-20T17:48:19.289Z,2024-12-20T19:53:53.536Z


In [11]:
print(len(df_applications['appName'].unique()))
print(len(df_applications['appPublisher'].unique()))

459
116


In [12]:
df_applications.to_csv('data/abs_apps.csv', index=False)